In [3]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr

from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from pandas import datetime
from pmdarima.arima import auto_arima

<ipython-input-3-c929ec775ba6>:10: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [4]:
path = './open'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))


In [5]:
# start_date = '20210101'
start_date = '20200607'
end_date = '20211031'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())
display(Business_days.tail())

sample_submission = pd.read_csv(os.path.join(path,sample_name))

WEEKDAY of "start_date" : 6
NUM of WEEKS to "end_date" : 43
HOW MANY "Business_days" : (365, 1)


,Date
0,2020-06-08
1,2020-06-09
2,2020-06-10
3,2020-06-11
4,2020-06-12


,Date
360,2021-10-25
361,2021-10-26
362,2021-10-27
363,2021-10-28
364,2021-10-29


In [6]:
# model = LinearRegression()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
    data.Close = data.Close.ffill()
    data.index = data["Date"]
    data.drop('Date', axis=1, inplace=True)
    
    

    
#     from statsmodels.tsa.stattools import adfuller
#     result = adfuller(data.Close)
#     print(result[0])
#     print("p-value", result[1])

#     plt.plot(data.Close)
#     plt.show()

#     plt.plot(data.Close.shift())
#     plt.show()
    
#     close_diff = data.Close - data.Close.shift()
#     plt.plot(close_diff)
#     plt.show()
    
    
#     result = adfuller(close_diff[1:])
#     print(result[0])
#     print(result[1])
    
#     import statsmodels.api as sm
    
#     fig = plt.figure(figsize=(20, 8))
#     ax1 = fig.add_subplot(212)
#     fig = sm.graphics.tsa.plot_acf(close_diff[1:], ax=ax1)
#     plt.show()

    
    # autoarima
    arima_data = data[:-1]
    model = auto_arima(arima_data.Close,
    start_p=0, start_q=0,d=1,
    max_p=4, max_q=4,max_d=2,
    m=5,
    start_P=0,start_Q=0,D=1,
    max_P=2,max_Q=2,max_D=2,
                           seasonal=True,
                           trace=False,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    model.fit(arima_data.Close)
    forecast = model.predict(n_periods=5) #(0,1,0)인수 나오는것 처리해야함
    predictions = list(forecast)
    print(predictions)
    # print(code)
    # break
    
#     ARIMA(p, d, q)
#     d=0 : ARMA(p, q), 정상성 만족
#     p=0 : IMA(d,q), d번 차분하면 MA(q)모형을 따르게 됨
#     q=0 : ARI(p,d), d번 차분한 시계열이 AR(p)모형을 따르게 됨

#     model = ARIMA(arima_data.Close, order=(4, 1, 0))
#     model_fit = model.fit()
    
#     start_index = datetime(2021, 11, 1)
#     end_index = datetime(2021, 11, 5)
#     forecast = model_fit.forecast(steps=5)
    
    
#     predictions = list(forecast)
    sample_submission.loc[0:4,code] = predictions 
sample_submission.isna().sum().sum()

  0%|          | 1/370 [00:16<1:40:41, 16.37s/it]

[70779.54356190517, 70626.00937177452, 70997.44784087701, 70546.1579610684, 70613.6611725553]


  1%|          | 2/370 [00:39<2:05:48, 20.51s/it]

[107603.67549241486, 108026.76878418593, 108940.46594206976, 108874.57376992254, 110686.27101148949]


  1%|          | 3/370 [00:56<1:53:52, 18.62s/it]

...

100%|█████████▉| 369/370 [3:13:51<00:16, 16.90s/it]

[43301.122592145846, 43629.24991318133, 44526.54464973909, 44377.16503249101, 43907.9144201846]


100%|██████████| 370/370 [3:14:10<00:00, 31.49s/it]

[49103.47968685174, 49826.86962481527, 49213.85400871794, 47623.76139579815, 47884.40592810321]


0

In [7]:
columns = list(sample_submission.columns[1:])

columns = ['Day'] + [str(x).zfill(6) for x in columns]

sample_submission.columns = columns

In [8]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,28662.309122,35495.534939,61006.053663,144351.415001,102443.460665,16846.857155,49434.506056,87006.945802,107603.675492,...,49055.111658,50627.950933,80386.475273,37496.397886,26452.614250,55586.372855,330365.168263,264253.319617,27534.549225,17807.780673
1,2021-11-02,28684.233081,36021.664244,60789.697309,144727.605182,105103.176909,16866.065141,49792.038491,88168.709847,108026.768784,...,49223.144096,50863.133342,82133.480629,37702.842839,26228.208138,55194.492037,330151.975805,262127.772236,27430.710930,17766.066366
2,2021-11-03,28985.835909,35835.462685,61117.956767,143808.357595,105717.791524,17082.524419,51011.988483,88441.029875,108940.465942,...,49054.837972,52341.097944,82701.533404,37939.982626,26842.222221,55546.807853,338527.255999,264640.701830,27516.367072,17740.304706
3,2021-11-04,29060.741007,35310.816998,60970.660650,143275.362703,106569.592850,16923.660172,50665.905341,88947.385554,108874.573770,...,48784.617744,52970.361140,83805.247443,37834.029245,26701.069934,55985.058408,342457.532738,268323.191955,27474.042192,18173.059011
4,2021-11-05,28806.836008,35086.990264,60468.836925,141648.028565,105237.551336,16823.623153,51250.901241,89290.295738,110686.271011,...,49821.425389,51512.288625,82391.141350,37634.794607,26195.855163,55893.060975,335329.366551,267117.938696,27455.416181,17794.850525
5,2021-11-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,2021-11-30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,2021-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,2021-12-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,2021-12-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:

start_date = '20211101'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )


real=[]
real_data=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\stock_predict2\\open\\sample_submission.csv')
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()

    real_data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    real_data_close_flatten = np.ravel(real_data_close.to_numpy())
    # print(real_data_close_flatten)
    

    real_data.loc[:4,code]=real_data_close_flatten


type(real_data)
type(sample_submission)

label_np = real_data.to_numpy()[0:5,1:]
pred_np = sample_submission.to_numpy()[0:5, 1:]
# print(pred_np)
print(label_np)
print(pred_np)
result=np.mean(abs(label_np-pred_np)/label_np)*100
print(result)

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (5, 1)


100%|██████████| 370/370 [04:42<00:00,  1.31it/s]

[[27850 35200 60000 ... 261600 26600 17600]
 [29250 35050 61700 ... 258600 27100 18100]
 [30250 34050 61500 ... 253700 26750 18300]
 [29450 33800 61100 ... 249600 26350 17550]
 [29550 33450 60600 ... 247800 26050 17550]]
[[28662.30912172709 35495.53493903899 61006.05366281918 ...
  264253.31961673714 27534.54922499878 17807.780672503348]
 [28684.233080883732 36021.66424364583 60789.697309361465 ...
  262127.77223622293 27430.710929919376 17766.06636608984]
 [28985.83590893711 35835.462685386614 61117.956766772644 ...
  264640.70183023653 27516.367071715544 17740.304706374474]
 [29060.741006670043 35310.81699814992 60970.660650097714 ...
  268323.19195525354 27474.042192488832 18173.059011081332]
 [28806.83600758845 35086.990264443244 60468.83692457952 ...
  267117.9386955889 27455.416181252458 17794.850525472237]]
3.8230251184324167


In [10]:

sample_submission.to_csv('arima.csv',index=False)

In [11]:
# start_date = '20210101'
start_date = '20200607'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())
display(Business_days.tail())


WEEKDAY of "start_date" : 6
NUM of WEEKS to "end_date" : 47
HOW MANY "Business_days" : (385, 1)


,Date
0,2020-06-08
1,2020-06-09
2,2020-06-10
3,2020-06-11
4,2020-06-12


,Date
380,2021-11-22
381,2021-11-23
382,2021-11-24
383,2021-11-25
384,2021-11-26


In [12]:
# model = LinearRegression()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
    
    data.Close = data.Close.ffill()
    data.index = data["Date"]
    data.drop('Date', axis=1, inplace=True)
    
    

    
#     from statsmodels.tsa.stattools import adfuller
#     result = adfuller(data.Close)
#     print(result[0])
#     print("p-value", result[1])

#     plt.plot(data.Close)
#     plt.show()

#     plt.plot(data.Close.shift())
#     plt.show()
    
#     close_diff = data.Close - data.Close.shift()
#     plt.plot(close_diff)
#     plt.show()
    
    
#     result = adfuller(close_diff[1:])
#     print(result[0])
#     print(result[1])
    
#     import statsmodels.api as sm
    
#     fig = plt.figure(figsize=(20, 8))
#     ax1 = fig.add_subplot(212)
#     fig = sm.graphics.tsa.plot_acf(close_diff[1:], ax=ax1)
#     plt.show()

    
    # autoarima
    arima_data = data[:-1]
    model = auto_arima(arima_data.Close,
    start_p=0, start_q=0,d=1,
    max_p=4, max_q=4,max_d=2,
    m=5,
    start_P=0,start_Q=0,D=1,
    max_P=2,max_Q=2,max_D=2,
                           seasonal=True,
                           trace=False,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    model.fit(arima_data.Close)
    forecast = model.predict(n_periods=5) #(0,1,0)인수 나오는것 처리해야함
    predictions = list(forecast)
    print(predictions)
    # print(code)
    # break
    
#     ARIMA(p, d, q)
#     d=0 : ARMA(p, q), 정상성 만족
#     p=0 : IMA(d,q), d번 차분하면 MA(q)모형을 따르게 됨
#     q=0 : ARI(p,d), d번 차분한 시계열이 AR(p)모형을 따르게 됨

#     model = ARIMA(arima_data.Close, order=(4, 1, 0))
#     model_fit = model.fit()
    
#     start_index = datetime(2021, 11, 1)
#     end_index = datetime(2021, 11, 5)
#     forecast = model_fit.forecast(steps=5)
    
    
#     predictions = list(forecast)
    sample_submission.loc[5:9,code] = predictions 
sample_submission.isna().sum().sum()

  0%|          | 1/370 [00:35<3:40:26, 35.84s/it]

[74112.70338369762, 75801.58543143269, 76063.58748107064, 75749.16196801816, 75087.87451319641]


  1%|          | 2/370 [01:11<3:38:14, 35.58s/it]

[118426.08436238616, 122223.92208829624, 122108.82239663323, 122343.11141181122, 121097.08751746855]


  1%|          | 3/370 [01:48<3:41:00, 36.13s/it]

...

[166741.36451056946, 165504.4349152488, 167339.25971654773, 166444.240574626, 171425.6738438253]


  9%|▊         | 32/370 [20:34<2:15:24, 24.04s/it]

[383599.6074202758, 381422.805615403, 382594.78707926575, 379241.510983015, 379626.6637886039]


  9%|▊         | 32/370 [20:47<3:39:38, 38.99s/it]


KeyboardInterrupt: 

In [ ]:
sample_submission.to_csv('arima.csv',index=False)